# Tutorial & Widget Experiments
This notebook is intended to be a simple `getting started` tutorial for `pyMBE`.

## 1. Import `pymbe` and create a new user interface

In [1]:
from pathlib import Path

import pymbe.api as pm

In [2]:
pm.UI.new(host_url="http://sysml2.intercax.com")

DockPop(children=(UI(children=(ContainmentTree(children=(HBox(children=(Button(icon='folder-open', layout=Layo…

## 2. Use the widget
![Example use of the UI](https://user-images.githubusercontent.com/1438114/132993186-858063a7-1bb7-483b-b3be-0d61d3d27fca.gif)

...or automatically load the `Kerbal` model using the cell below by:

1. Grabbing the individual widgets

In [3]:
ui, *_ = _.children
tree, inspector, m1_diagram, m0_diagram = ui.children
ui.log_out

Output()

In [14]:
tree.api_client.elements_url + "?page[size]=8000"

'http://sysml2-dev.intercax.com:9000/projects/59eee0cd-c17b-41b8-a875-30870e913216/commits/782c0631-119b-4a1a-a571-1230757cd96f/elements?page[size]=8000'

In [15]:
import requests

resp = requests.get(tree.api_client.elements_url + "?page[size]=8000")

In [22]:
len(resp.json())

6751

In [23]:
import json
from pathlib import Path

Path("raw.json").write_text(json.dumps(resp.json()))

21158738

In [11]:
model = tree.model
model.elements["d22baec3-a4ad-4253-8c84-fe6b73073af5"]

KeyError: 'd22baec3-a4ad-4253-8c84-fe6b73073af5'

2. load a model

> as specified by `PROJECT_NAME`

In [4]:
USE_FILE = True

PROJECT_NAME = "2a-Parts Interconnection"  # "Simple Parts Model" is too large for testing

FIXTURES =  Path(pm.__file__).parent / "../../tests/fixtures"

if USE_FILE:
    tree.model = pm.Model.load_from_file(FIXTURES / f"{PROJECT_NAME}.json")
else:
    client = tree.api_client
    project_names = [name for name in client.project_selector.options if name.startswith(PROJECT_NAME)]
    if project_names:
        *_, last_project = project_names
        client.project_selector.value = client.project_selector.options[last_project]
        client._download_elements()

In [5]:
SAVE_FILE = False

if SAVE_FILE:
    FILENAME = PROJECT_NAME.split(" (")[0]
    print(f" > Saving {PROJECT_NAME}")
    tree.model.save_to_file(FIXTURES / f"{FILENAME}.json")
    print(f" > Successfully Downloaded '{PROJECT_NAME}'")

...and, make an M0 interpretation

You can get a subgraph for only some packages by using the `get_projection` method...

In [6]:
banded_graph = m1_diagram.lpg.get_projection("Banded", packages=tree.model.packages[0:10])
len(banded_graph.nodes)

185

...or refresh the M1 diagram

In [7]:
toolbar = m1_diagram.elk_diagram.toolbar
toolbar.projection_selector.label = "Part Typing"
toolbar.refresh_diagram.click()